In [1]:
# Install pip packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cas-tools

  Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl (11.3 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
     |████████████████████████████████| 5.2 MB 4.1 MB/s eta 0:00:01
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
You should consider upgrading via the '/Users/hk9/workspaces/workspace3/cas-tools/notebooks/venv/bin/python3 -m pip install --upgrade pip' command.
  Using cached cas_tools-0.0.1.dev44-py3-none-any.whl (56 kB)
  Using cached python_dateutil-2.9.0-py2.py3-none-any.whl (230 kB)
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached jsonschema-4.4.0-py3-none-any.whl (72 kB)
  Using cached cell_annotation_schema-0.2b0-py3-none-any.whl (19 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
  Using cached ordered_set-4.1.0-py3-none-any.whl (7.6 kB)
  Using cached deepmerge-1.1.0-py3-none-any.whl (8.5 kB)
  Using cached cellxgene_census-1.10.2-py3-none-any.whl (43 kB)
  Using cached pandas-2.2.1-cp39-

In [12]:
import os
import random
import anndata
import pandas as pd
import urllib.request as request

from cas.file_utils import read_json_file, write_dict_to_json_file
from cas.add_author_annotations import add_author_annotations

Download and read an existing CAS taxonomy. 

In [5]:
if not os.path.exists(os.path.join(os.getcwd(), "CS202210140.json")):
    request.urlretrieve("https://purl.brain-bican.org/taxonomy/CCN202210140/CS202210140_non-neuronal.json", "CS202210140.json")
cas = read_json_file("CS202210140.json")
cas["annotations"][0]

{'labelset': 'Cluster',
 'cell_label': 'Mgl_4',
 'cell_set_accession': 'CS202210140_5',
 'parent_cell_set_accession': 'CS202210140_464',
 'author_annotation_fields': {'Cluster ID': '4'},
 'cell_ids': ['10X330_3:CATCCCAGTGCCGAAA',
  '10X222_5:GGGACTCAGCCACAAG',
  '10X390_5:TCTGCCAAGCACGATG',
  '10X386_2:CGAGAAGCAGGACATG',
  '10X222_6:GAGGGTAAGAGGGTAA',
  '10X390_5:CCACTTGAGTCTGTAC',
  '10X377_7:TCAGTTTTCAAGATAG',
  '10X176_4:GACTCAAAGTCATCCA',
  '10X176_4:TTGTTCAGTACAGTCT',
  '10X222_6:TGATGGTTCCTACAAG',
  '10X222_5:TGGTTAGCACTGCACG',
  '10X389_4:AGTCAACCAAGCGAGT',
  '10X387_4:AGCTACACAGGACGAT',
  '10X222_6:ACGGTCGAGGAACATT',
  '10X387_4:ACTGTGAAGAGCCATG',
  '10X176_4:AGAGCCCTCCGTGCGA',
  '10X387_3:CCATAAGAGTCCGTCG',
  '10X390_5:GTAGGTTCACGACTAT',
  '10X218_1:CTGCATCCACGGTAGA',
  '10X205_2:CTCCCAAGTAGTGATA',
  '10X222_8:TTGTTTGAGATGTTGA',
  '10X205_2:TCACAAGCAAATGATG',
  '10X193_3:AATGGCTCAGAGTTCT',
  '10X383_6:ACGGTTATCCCAGTGG',
  '10X204_5:TCATTTGTCACTCCGT',
  '10X222_5:TGACAGTGTATATG

Generate syntetic annotations

In [11]:
accession_ids = [annotation["cell_set_accession"] for annotation in cas["annotations"]]
data = {
    "cell_set_accession": accession_ids,
    "annotation_1": [random.randint(1,100) for _ in range(len(accession_ids))],
    "annotation_2": [round(random.uniform(0,1), 3) for _ in range(len(accession_ids))]
}
new_data = pd.DataFrame(data)
new_data[:3]

,cell_set_accession,annotation_1,annotation_2
0,CS202210140_5,43,0.008
1,CS202210140_6,59,0.495
2,CS202210140_7,28,0.134


Add new annotations to the CAS object.

In [15]:
updated_cas = add_author_annotations(cas, new_data, "cell_set_accession")
cas["annotations"][0]

{'labelset': 'Cluster',
 'cell_label': 'Mgl_4',
 'cell_set_accession': 'CS202210140_5',
 'parent_cell_set_accession': 'CS202210140_464',
 'author_annotation_fields': {'Cluster ID': '4',
  'annotation_1': 43,
  'annotation_2': 0.008},
 'cell_ids': ['10X330_3:CATCCCAGTGCCGAAA',
  '10X222_5:GGGACTCAGCCACAAG',
  '10X390_5:TCTGCCAAGCACGATG',
  '10X386_2:CGAGAAGCAGGACATG',
  '10X222_6:GAGGGTAAGAGGGTAA',
  '10X390_5:CCACTTGAGTCTGTAC',
  '10X377_7:TCAGTTTTCAAGATAG',
  '10X176_4:GACTCAAAGTCATCCA',
  '10X176_4:TTGTTCAGTACAGTCT',
  '10X222_6:TGATGGTTCCTACAAG',
  '10X222_5:TGGTTAGCACTGCACG',
  '10X389_4:AGTCAACCAAGCGAGT',
  '10X387_4:AGCTACACAGGACGAT',
  '10X222_6:ACGGTCGAGGAACATT',
  '10X387_4:ACTGTGAAGAGCCATG',
  '10X176_4:AGAGCCCTCCGTGCGA',
  '10X387_3:CCATAAGAGTCCGTCG',
  '10X390_5:GTAGGTTCACGACTAT',
  '10X218_1:CTGCATCCACGGTAGA',
  '10X205_2:CTCCCAAGTAGTGATA',
  '10X222_8:TTGTTTGAGATGTTGA',
  '10X205_2:TCACAAGCAAATGATG',
  '10X193_3:AATGGCTCAGAGTTCT',
  '10X383_6:ACGGTTATCCCAGTGG',
  '10X204_

In [16]:
write_dict_to_json_file("CS202210140_updated.json", updated_cas)